In [2]:
import numpy as np
import torch
bikes_numpy = np.loadtxt("hour-fixed.csv",
                         dtype=np.float32,
                         delimiter=',',
                         skiprows=1,
                         converters={1: lambda x: float(x[8:10])}
                        )
bikes = torch.from_numpy(bikes_numpy)
bikes

tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 3.0000e+00, 1.3000e+01,
         1.6000e+01],
        [2.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 8.0000e+00, 3.2000e+01,
         4.0000e+01],
        [3.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 5.0000e+00, 2.7000e+01,
         3.2000e+01],
        ...,
        [1.7377e+04, 3.1000e+01, 1.0000e+00,  ..., 7.0000e+00, 8.3000e+01,
         9.0000e+01],
        [1.7378e+04, 3.1000e+01, 1.0000e+00,  ..., 1.3000e+01, 4.8000e+01,
         6.1000e+01],
        [1.7379e+04, 3.1000e+01, 1.0000e+00,  ..., 1.2000e+01, 3.7000e+01,
         4.9000e+01]])

In [3]:
bikes.shape  # 2 years * 365 days * 24 hours = 17520 hours       17 attributes

torch.Size([17520, 17])

In [4]:
daily_bikes = bikes.view(-1, 24, bikes.shape[1])  # 第1軸和第2軸給定前提, 第0軸電腦自己產生  17520 / 24 = 730
daily_bikes.shape

torch.Size([730, 24, 17])

In [5]:
daily_bikes = daily_bikes.transpose(1, 2)
daily_bikes.shape

torch.Size([730, 17, 24])

In [6]:
first_day = bikes[:24].long()    #第一個24小時
weather_onehot = torch.zeros(24, 4)    # 4種天氣等級 * 24小時
first_day[:, 9]   #第一天第9行的資訊(天氣狀況)

tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2])

In [18]:
first_day[:,9].unsqueeze(1).long() - 1  #-1是因為index 0開始

tensor([[0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [2],
        [2],
        [1],
        [1],
        [1],
        [1]])

In [7]:
weather_onehot.scatter_(1, first_day[:,9].unsqueeze(1).long() - 1, 1.0)

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

In [9]:
torch.cat((bikes[:24], weather_onehot), 1)[:1]   #one-hot資料和原資料連結 (沿第一軸)

tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  6.0000,
          0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,  3.0000, 13.0000,
         16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

In [13]:
daily_weather_onehot = torch.zeros(daily_bikes.shape[0], 4, daily_bikes.shape[2]) # 730 days, 4 天氣狀況, 24 hours  創建one hot tensor
daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [14]:
daily_weather_onehot.scatter_(1, daily_bikes[:, 9, :].long().unsqueeze(1) - 1, 1.0) #one hot encoding
daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [16]:
daily_bikes = torch.cat((daily_bikes, daily_weather_onehot), dim=1)  #與原資料連結

In [17]:
daily_bikes[:, 9, :] = (daily_bikes[:, 9, :] - 1.0) / 3.0   #將天氣狀況投影至0~1

In [19]:
#對攝氏溫度做正規化
temp = daily_bikes[:, 10, :]
temp_min = torch.min(temp)
temp_max = torch.max(temp)
daily_bikes[:, 10,:] = (daily_bikes[:, 10, :] - temp_min) / (temp_max - temp_min)
daily_bikes[:, 10,:]

tensor([[0.2245, 0.2041, 0.2041,  ..., 0.3878, 0.3878, 0.4490],
        [0.4490, 0.4286, 0.4082,  ..., 0.2449, 0.2245, 0.2041],
        [0.2041, 0.1837, 0.1837,  ..., 0.1633, 0.1224, 0.1633],
        ...,
        [0.2245, 0.2245, 0.2245,  ..., 0.2653, 0.2449, 0.2449],
        [0.2449, 0.2449, 0.2449,  ..., 0.1837, 0.1837, 0.1837],
        [0.1633, 0.1633, 0.1429,  ..., 0.2449, 0.2449, 0.2449]])

In [20]:
temp = daily_bikes[:, 10,:]
daily_bikes[:, 10,:] = (daily_bikes[:, 10,:] - torch.mean(temp)) / torch.std(temp)
daily_bikes[:, 10,:]

tensor([[-1.3213, -1.4248, -1.4248,  ..., -0.4932, -0.4932, -0.1827],
        [-0.1827, -0.2862, -0.3897,  ..., -1.2178, -1.3213, -1.4248],
        [-1.4248, -1.5284, -1.5284,  ..., -1.6319, -1.8389, -1.6319],
        ...,
        [-1.3213, -1.3213, -1.3213,  ..., -1.1143, -1.2178, -1.2178],
        [-1.2178, -1.2178, -1.2178,  ..., -1.5284, -1.5284, -1.5284],
        [-1.6319, -1.6319, -1.7354,  ..., -1.2178, -1.2178, -1.2178]])